In [3]:
from PIL import Image, ImageFont, ImageDraw 
import requests
import io


In [4]:
def get_google_font(font: str, size=24):
    """
    Questa funzione scarica temporaneamente da github.com/google/fonts/ il file .ttf del font desiderato.
    font: stringa indicante il percorso del font su github.com/google/fonts/ nel formato cartella/font_file.ttf
    size: dimensione del font
    """
    url = f"https://github.com/google/fonts/blob/main/ofl/{font}?raw=true"
    print(url)
    r = requests.get(url, allow_redirects=True)
    font = ImageFont.truetype(io.BytesIO(r.content), size=size)

    return font

def crop_image(my_image, aspect_ratio: tuple):
    image  = my_image
    width  = image.size[0]
    height = image.size[1]

    aspect_now = width / float(height)

    ideal_width = aspect_ratio[0]
    ideal_height = aspect_ratio[1]
    ideal_aspect = ideal_width / float(ideal_height)

    if aspect_now > ideal_aspect:
        # Then crop the left and right edges:
        new_width = int(ideal_aspect * height)
        offset = (width - new_width) / 2
        resize = (offset, 0, width - offset, height)
    else:
        # ... crop the top and bottom:
        new_height = int(width / ideal_aspect)
        offset = (height - new_height) / 2
        resize = (0, offset, width, height - offset)

    image = image.crop(resize).resize((ideal_width, ideal_height), Image.LANCZOS)
    # img_path_cropped = "CROPPED_"+img_path
    # thumb.save(img_path_cropped)
    return image

def write_on_image(my_image, edit_dict):
    image_editable = ImageDraw.Draw(my_image)

    for scope in edit_dict.keys():
        # Get textbox dimensions from coordinates
        box_w = edit_dict[scope]["textbox"][1][0] - edit_dict[scope]["textbox"][0][0]
        box_h = edit_dict[scope]["textbox"][1][1] - edit_dict[scope]["textbox"][0][1]

        # Get appropriate font size to make the text fit the box in a single line
        max_size = edit_dict[scope]["size"] # Start from default font size
        text_length = get_google_font(edit_dict[scope]["font"], max_size).getlength(edit_dict[scope]["text"]) # Get text length given font and size
        # Finché il testo non ci sta nel box predefinito, riduco la size di 2 punti e riprovo
        while text_length > box_w:
            max_size -= 2
            text_length = get_google_font(edit_dict[scope]["font"], max_size).getlength(edit_dict[scope]["text"])

        image_editable.text(edit_dict[scope]["position"], 
                            edit_dict[scope]["text"],
                            edit_dict[scope]["color"],
                            anchor=edit_dict[scope]["anchor"],
                            font=get_google_font(edit_dict[scope]["font"]
                                                    , max_size))
    
    return my_image

def image_edit(img_path: str, edit_dict: dict, crop=True):
    
    if crop:
        # Carico immagine
        my_image = Image.open(img_path) 
        # Crop immagine
        my_image = crop_image(my_image, (648,1152))
    else:
        # Carico immagine
        my_image = Image.open(img_path) 

    my_image = write_on_image(my_image, edit_dict)

    # Salvataggio risutlato
    my_image.save("EDIT_"+img_path)

In [5]:
img_path = "test_img.png"

# da fare:
# - mettere posizione parametrica (e.g. titolo centrato)
# https://stackoverflow.com/questions/1970807/center-middle-align-text-with-pil
# https://pillow.readthedocs.io/en/stable/handbook/text-anchors.html
# - scegliere colore testo parametrico, adatto all'immagine


edit_dict = {
    "title": {"text": "Torneo di Bang",
                "font": "montserratalternates/MontserratAlternates-Bold.ttf",
                "size": 90, 
                "color": (55, 0, 134),
                "position":(324,100),
                "anchor":"mm"},
    "body": {"text": "",
                "font": "montserratalternates/MontserratAlternates-Bold.ttf",
                "size": 80, 
                "color": (55, 0, 134),
                "position":(0,0),
                "anchor":"mm"},
    "location": {"text": "Centro NOI di Povegliano",
                "font": "montserratalternates/MontserratAlternates-Bold.ttf",
                "size": 40, 
                "color": (55, 0, 134),
                "position":(0,900),
                "anchor":"lb"},
    "time": {"text": "Mercoledì 31 febbraio ore 21:00",
                "font": "montserratalternates/MontserratAlternates-Bold.ttf",
                "size": 40, 
                "color": (55, 0, 134),
                "position":(0,950),
                "anchor":"lb"},
    "time": {"text": "21:00",
                "font": "montserratalternates/MontserratAlternates-Bold.ttf",
                "size": 40, 
                "color": (55, 0, 134),
                "position":(0,950),
                "anchor":"lb"},
    "cost": {"text": "Iscrizione libera",
                "font": "montserratalternates/MontserratAlternates-Bold.ttf",
                "size": 40, 
                "color": (55, 0, 134),
                "position":(0,1000),
                "anchor":"lb"}
}

img_path_edit = image_edit(img_path, edit_dict)

https://github.com/google/fonts/blob/main/ofl/montserratalternates/MontserratAlternates-Bold.ttf?raw=true
https://github.com/google/fonts/blob/main/ofl/montserratalternates/MontserratAlternates-Bold.ttf?raw=true
https://github.com/google/fonts/blob/main/ofl/montserratalternates/MontserratAlternates-Bold.ttf?raw=true
https://github.com/google/fonts/blob/main/ofl/montserratalternates/MontserratAlternates-Bold.ttf?raw=true
https://github.com/google/fonts/blob/main/ofl/montserratalternates/MontserratAlternates-Bold.ttf?raw=true


In [9]:
img = Image.open("test_img.png")
draw = ImageDraw.Draw(img)
font = get_google_font("montserratalternates/MontserratAlternates-Bold.ttf"
                                                    , 80)

# width,height= draw.textsize("Sample Text", font=font)
draw.multiline_text((100, 100),"Sample Text Sample Text Sample Text Sample Text",(255,255,255),font=font, align='center')
img.save('sample-out.jpg')

https://github.com/google/fonts/blob/main/ofl/montserratalternates/MontserratAlternates-Bold.ttf?raw=true


In [13]:
from PIL import Image, ImageDraw, ImageFont
import textwrap

astr = '''The rain in Spain falls mainly on the plains.'''
para = textwrap.wrap(astr, width=15)

MAX_W, MAX_H = 648,1152
img = Image.open("test_img.png")
img = crop_image(img, (648,1152))
draw = ImageDraw.Draw(img)
font = get_google_font("montserratalternates/MontserratAlternates-Bold.ttf"
                                                    , 80)

current_h, pad = 50, 10
for line in para:
    w, h = draw.textlength(line, font=font), 80
    draw.text(((MAX_W - w) / 2, current_h), line, font=font)
    current_h += h + pad

img.save('test.png')

https://github.com/google/fonts/blob/main/ofl/montserratalternates/MontserratAlternates-Bold.ttf?raw=true


In [1]:
int(1.5)

1

In [22]:
from PIL import Image, ImageDraw, ImageFont
import textwrap
img = Image.open("test_img.png")
img = crop_image(img, (648,1152))

draw = ImageDraw.Draw(img)
draw.rectangle(((50, 30), (600, 170)), width=3) # TITOLO
draw.rectangle(((50, 170), (600, 270)), width=3) # BODY
draw.rectangle(((50, 860), (324, 920)), width=3) # DATE
draw.rectangle(((50, 920), (324, 980)), width=3) # TIME
draw.rectangle(((50, 980), (324, 1040)), width=3) # LOCATION
draw.rectangle(((50, 1040), (324, 1100)), width=3) # COST



img.save('layout_example.png')